# Chatbot Tensorflow
> Attention Mechanism을 적용한 seq2seq 모델과 Tensorflow, Keras로 제작한 Chatbot 튜토리얼입니다.

### Dataset Download
Chatbot 학습에 필요한 Dataset을 불러옵니다.
- [songys/Chatbot_data](https://github.com/songys/Chatbot_data.git)
- 문답 페어 11,876개
- `Q`: 질문
- `A`: 답변

In [1]:
!git clone https://github.com/songys/Chatbot_data.git

Cloning into 'Chatbot_data'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 50 (delta 17), reused 2 (delta 1), pack-reused 18
Unpacking objects: 100% (50/50), done.


In [2]:
import pandas as pd
corpus = pd.read_csv('/content/Chatbot_data/ChatbotData.csv')

In [3]:
corpus.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
# Question data
corpus.Q.head()

0             12시 땡!
1        1지망 학교 떨어졌어
2       3박4일 놀러가고 싶다
3    3박4일 정도 놀러가고 싶다
4            PPL 심하네
Name: Q, dtype: object

In [5]:
# Answer data
corpus.A.head()

0     하루가 또 가네요.
1      위로해 드립니다.
2    여행은 언제나 좋죠.
3    여행은 언제나 좋죠.
4     눈살이 찌푸려지죠.
Name: A, dtype: object

In [6]:
# dataset의 type을 list 형태로 변환
q_list = []
a_list = []

for q, a in zip(corpus.Q, corpus.A):
    q_list.append(q)
    a_list.append(a)

In [7]:
# RAM 용량 제한으로 인한 데이터 개수 조정
q_list = q_list[:3000]
a_list = a_list[:3000]

In [8]:
q_list[:5]

['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']

### Preprocess
형태소 분석
- Konlpy의 Okt 분석기를 사용합니다.
   

토큰 추가
- `SOS`: Start Of Sentence
- `EOS`: End Of Sentence

In [9]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 222 kB/s 
     |████████████████████████████████| 448 kB 71.5 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [10]:
from konlpy.tag import Okt
okt = Okt()

In [11]:
sentence = "오늘은 즐거운 자연어처리를 해보았어요"
okt.morphs(sentence)

['오늘', '은', '즐거운', '자연어', '처리', '를', '해보았어요']

In [12]:
# 형태소 분석으로 분할된 단어들을 공백 기준으로 분리
def process_morph(sentence):
    return ' '.join(okt.morphs(sentence))

In [13]:
# 질문과 답변을 분리해서 형태소 분석 및 토큰 추가
def morph_and_token(sentence, is_question=True):
    sentence = process_morph(sentence)
    if is_question:
        return sentence
    else:
        return ('<SOS> ' + sentence, sentence + ' <EOS>')

In [14]:
def preprocess(q_list, a_list):
    questions = []
    answer_input = []
    answer_output = []

    for q in q_list:
        question = morph_and_token(q, is_question=True)
        questions.append(question)

    for a in a_list:
        input_, output_ = morph_and_token(a, is_question=False)
        answer_input.append(input_)
        answer_output.append(output_)

    return questions, answer_input, answer_output

### Dataset Split
Encoder, Decoder의 관점으로 Dataset을 재구성합니다.
- `questions`: Encoder input  
- `answer_input`: Decoder input  
- `answer_output`: Decoder output

In [15]:
questions, answer_input, answer_output = preprocess(q_list, a_list)

In [16]:
questions[:5]

['12시 땡 !', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다', '3 박 4일 정도 놀러 가고 싶다', 'PPL 심하네']

In [17]:
answer_input[:5]

['<SOS> 하루 가 또 가네요 .',
 '<SOS> 위로 해 드립니다 .',
 '<SOS> 여행 은 언제나 좋죠 .',
 '<SOS> 여행 은 언제나 좋죠 .',
 '<SOS> 눈살 이 찌푸려지죠 .']

In [18]:
answer_output[:5]

['하루 가 또 가네요 . <EOS>',
 '위로 해 드립니다 . <EOS>',
 '여행 은 언제나 좋죠 . <EOS>',
 '여행 은 언제나 좋죠 . <EOS>',
 '눈살 이 찌푸려지죠 . <EOS>']

In [19]:
# vocab 제작에 사용
all_sentences = questions + answer_input + answer_output
all_sentences[:5]

['12시 땡 !', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다', '3 박 4일 정도 놀러 가고 싶다', 'PPL 심하네']

### Tokenization
- Vocab을 만들어줍니다.
- Text를 Sequence로 Encoding합니다.
- Padding으로 문장의 길이를 일정하게 맞춰줍니다.

In [20]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [21]:
# 토큰의 옵션 정의
# OOV는 Out Of Vocabulary
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')

In [22]:
# internal vocabulary 생성
tokenizer.fit_on_texts(all_sentences)

In [23]:
VOCAB_SIZE = len(tokenizer.word_index) + 1

In [24]:
VOCAB_SIZE

4672

In [25]:
# vocab 확인해보기
for word, index in tokenizer.word_index.items():
    print(f'{index}\t\t\t{word}')
    if index == 10:
        break

1			<OOV>
2			.
3			<SOS>
4			<EOS>
5			이
6			거
7			을
8			가
9			예요
10			도


In [26]:
# Text to Sequence Encoding
questions_sequence = tokenizer.texts_to_sequences(questions)
answer_input_sequence = tokenizer.texts_to_sequences(answer_input)
answer_output_sequence = tokenizer.texts_to_sequences(answer_output)

In [27]:
# Vocab에 저장되어있다면 index를 반환
tokenizer.word_index['곱창']

2466

In [28]:
questions[:5]

['12시 땡 !', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다', '3 박 4일 정도 놀러 가고 싶다', 'PPL 심하네']

In [29]:
questions_sequence[:5]

[[2418, 3256, 27],
 [1353, 3257, 3258, 1354],
 [1355, 2419, 2420, 282, 156, 78],
 [1355, 2419, 2420, 2421, 282, 156, 78],
 [3259, 3260]]

In [30]:
# Padding Hyperparameter
MAX_LENGTH = 30

In [31]:
# post -> 문장을 잘라낼때 뒷부분부터 잘라주고, Padding을 해줄때 뒷부분부터 채워넣음
questions_padded = pad_sequences(questions_sequence, maxlen=MAX_LENGTH, padding='post', truncating='post')
answer_input_padded = pad_sequences(answer_input_sequence, maxlen=MAX_LENGTH, padding='post', truncating='post')
answer_output_padded = pad_sequences(answer_output_sequence, maxlen=MAX_LENGTH, padding='post', truncating='post')

In [32]:
questions_padded[:5]

array([[2418, 3256,   27,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [1353, 3257, 3258, 1354,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [1355, 2419, 2420,  282,  156,   78,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [1355, 2419, 2420, 2421,  282,  156,   78,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [3259, 3260,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 

In [33]:
questions_padded.shape

(3000, 30)

### Vectorization
- 각 단어들을 One-Hot Encoding 변환
- Vocab의 index를 참조해 다시 text 형태로 변환 (예측 과정에서 호출)

In [34]:
#One-Hot Encoding
def convert_to_one_hot(padded):
    one_hot_vector = np.zeros((len(padded), MAX_LENGTH, VOCAB_SIZE))

    for i, sequence in enumerate(padded):
        for j, index in enumerate(sequence):
            one_hot_vector[i, j, index] = 1
    
    return one_hot_vector

In [35]:
#answer_input_one_hot = convert_to_one_hot(answer_input_padded)
answer_output_one_hot = convert_to_one_hot(answer_output_padded)

In [36]:
answer_output_one_hot.shape

(3000, 30, 4672)

In [37]:
len(answer_output_one_hot[0][0])

4672

In [38]:
# 예측 값을 단어사전에서 찾아와 문자열로 변환
def index_to_text(indexs, end_token):
    sentence = ' '

    for i in indexs:
        if i == end_token:
            break;

        if i > 0 and tokenizer.index_word[i] is not None:
            sentence += tokenizer.index_word[i]
        else:
            sentence += ''

        sentence += ' '
    return sentence

### Generate Model
- Encoder 정의
- Decoder 정의
- Seq2Seq에 Attention Mechanism 적용

In [39]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Attention
from keras.models import Model

In [40]:
# tf.keras.Model을 상속받아 Encoder 함수를 정의합니다.
class Encoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        # tf.keras.Model의 init함수를 호출합니다.
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim, input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units, return_sequences=True, return_state=True)

    def __call__(self, inputs):
        x = self.embedding(inputs)
        x = self.dropout(x)
        H, hidden_state, cell_state = self.lstm(x)
        # return all Hidden states and context vector
        return H, hidden_state, cell_state

In [41]:
class Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Decoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim, input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units, return_state=True, return_sequences=True, )
        self.attention = Attention()
        self.dense = Dense(vocab_size, activation='softmax')

    def __call__(self, inputs, initial_state):
        # H는 Encoder 내부에 있는 모든 LSTM의 hidden state
        x, h0, c0, H = inputs
        x = self.embedding(x)
        x = self.dropout(x)
        # S는 Decoder 내부에 있는 모든 LSTM의 hidden state
        S, hidden_state, cell_state = self.lstm(x, initial_state=[h0, c0])
        S_ = tf.concat([h0[:, tf.newaxis, :], S[:, :-1, :]], axis=1)

        A = self.attention([S_, H])
        y = tf.concat([S, A], axis=-1)
        y = self.dense(y)
        return y, hidden_state, cell_state

In [42]:
class Seq2Seq(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
        super(Seq2Seq, self).__init__()
        self.start_token = start_token
        self.end_token = end_token
        self.time_steps = time_steps

        self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
        self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)

    def __call__(self, inputs, training=True):
        # 학습 상태의 경우
        if training:
            encoder_inputs, decoder_inputs = inputs
            #decoder_outputs = decoder_inputs
            H, decoder_hidden, decoder_cell = self.encoder(encoder_inputs)
            decoder_outputs, _, _ = self.decoder(inputs=[decoder_inputs, decoder_hidden, decoder_cell, H], initial_state=[decoder_inputs, decoder_hidden, decoder_cell, H])
            return decoder_outputs

        # 예측 상태의 경우
        else:
            H, decoder_hidden, decoder_cell = self.encoder(inputs)
            target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
            results = tf.TensorArray(tf.int32, self.time_steps)

            # 맨 처음 한번만 <SOS> 토큰을 넣어줍니다.
            decoder_outputs = target_seq
            for i in tf.range(self.time_steps):
                decoder_outputs, decoder_hidden, decoder_cell = self.decoder(inputs=[decoder_outputs, decoder_hidden, decoder_cell, H], initial_state=[decoder_outputs, decoder_hidden, decoder_cell, H])
                decoder_outputs = tf.cast(tf.argmax(decoder_outputs, axis=-1), dtype=tf.int32)
                decoder_outputs = tf.reshape(decoder_outputs, shape=(1, 1))
                results = results.write(i, decoder_outputs)

                if decoder_outputs == self.end_token:
                    break;

                #target_seq = decoder_outputs
                #context_vector = [decoder_hidden, decoder_cell]

            return tf.reshape(results.stack(), shape=(1, self.time_steps))

### Train

In [43]:
# Training Hyperparameter
BATCH_SIZE = 16
EMBEDDING_DIM = 128
TIME_STEPS = MAX_LENGTH

START_TOKEN = tokenizer.word_index['<SOS>']
END_TOKEN = tokenizer.word_index['<EOS>']

# LSTM에 들어가는 UNITS
UNITS = 128

VOCAB_SIZE = len(tokenizer.word_index) + 1
DATA_LENGTH = len(questions)
SAMPLE_SIZE = 3
CALL_NUM = 20

In [44]:
# 모델 생성 및 컴파일
seq2seq = Seq2Seq(UNITS, VOCAB_SIZE, EMBEDDING_DIM, TIME_STEPS, START_TOKEN, END_TOKEN)
seq2seq.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [45]:
# 예측 함수
def make_prediction(model, question_inputs):
    results = model(inputs=question_inputs, training=False)
    results = np.asarray(results).reshape(-1)
    return results

In [46]:
# 주어진 epoch만큼 모델 학습
for cnt in range(CALL_NUM):
    print(f'processing epoch: {cnt * 10 + 1}...')
    seq2seq.fit([questions_padded, answer_input_padded],
                answer_output_one_hot,
                batch_size=BATCH_SIZE,
                epochs=10,
                )
    
    samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

    for idx in samples:
        question_inputs = questions_padded[idx]
        results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))
        results = index_to_text(results, END_TOKEN)

        print(f'Q: {questions[idx]}')
        print(f'A: {results}\n')
        print()

processing epoch: 1...
Epoch 1/10
188/188 [==============================] - 8s 14ms/step - loss: 1.9170 - acc: 0.7862
Epoch 2/10
188/188 [==============================] - 3s 14ms/step - loss: 1.1343 - acc: 0.8374
Epoch 3/10
188/188 [==============================] - 3s 14ms/step - loss: 1.0550 - acc: 0.8411
Epoch 4/10
188/188 [==============================] - 3s 14ms/step - loss: 1.0070 - acc: 0.8456
Epoch 5/10
188/188 [==============================] - 3s 14ms/step - loss: 0.9662 - acc: 0.8491
Epoch 6/10
188/188 [==============================] - 3s 14ms/step - loss: 0.9287 - acc: 0.8524
Epoch 7/10
188/188 [==============================] - 3s 14ms/step - loss: 0.8881 - acc: 0.8558
Epoch 8/10
188/188 [==============================] - 3s 14ms/step - loss: 0.8460 - acc: 0.8602
Epoch 9/10
188/188 [==============================] - 3s 14ms/step - loss: 0.8044 - acc: 0.8639
Epoch 10/10
188/188 [==============================] - 3s 14ms/step - loss: 0.7630 - acc: 0.8690
Q: 너 도 고민 있어 ?
A

### Prediction
- 사용자로부터 입력받은 문장의 전처리를 해줍니다.
- 전처리 한 문장을 입력해 예측값을 얻습니다.

In [47]:
# 입력받은 문장을 전처리
def make_question(sentence):
    sentence = morph_and_token(sentence)
    question_sequence = tokenizer.texts_to_sequences([sentence])
    question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
    return question_padded

In [48]:
make_question('1지망 학교 떨어졌어')

array([[1353, 3257, 3258, 1354,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)

In [49]:
# make_question으로 전처리 후 make_prediction으로 예측
def run_chatbot(question):
    question_inputs = make_question(question)
    results = make_prediction(seq2seq, question_inputs)
    results = index_to_text(results, END_TOKEN)
    return results

### Test
- 챗봇과 대화를 이어갈 수 있는 코드

In [50]:
EXIT = "대화종료"
input_values = []

while True:
    user_input = input('\nQuestion: ')
    if user_input == EXIT:
        break
    input_values.append(user_input)
    answer = run_chatbot(user_input)
    print(f'Answer: {answer}')
    print('---------------------------')


Question: 비 오는 날 별로 안 좋아해
Answer:  너무 아름답죠 . 
---------------------------

Question: 배고파
Answer:  얼른 맛 난 음식 드세요 . 
---------------------------

Question: 나 너무 지쳤어
Answer:  지칠 때 는 쉬어도 돼요 . 
---------------------------

Question: 하고싶은게 너무 많아
Answer:  잘 하고 있을 거 예요 . 
---------------------------

Question: 고마워
Answer:  저 도 보고 싶어요 . 
---------------------------

Question: 보고싶지는 않아
Answer:  저 는 주 당 이에요 . 
---------------------------

Question: 오늘은 기분이 너무 좋아
Answer:  감기 조심하세요 . 
---------------------------

Question: 날씨가 좋다
Answer:  그게 최고 죠 . 
---------------------------

Question: 보고싶은 영화가 많아
Answer:  나중 에 없애주세요 . 
---------------------------

Question: 대화종료


In [51]:
input_values

['비 오는 날 별로 안 좋아해',
 '배고파',
 '나 너무 지쳤어',
 '하고싶은게 너무 많아',
 '고마워',
 '보고싶지는 않아',
 '오늘은 기분이 너무 좋아',
 '날씨가 좋다',
 '보고싶은 영화가 많아']